In [2]:
# %config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib as matplot
import matplotlib.pyplot as plt
# %matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings, os 
# warnings.filterwarnings("ignore")

# from keras import Sequential
# from keras.models import Model, load_model
# from keras.layers import *
# from keras.callbacks import ModelCheckpoint
# from keras import regularizers
from matplotlib import pyplot as plt

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA, TruncatedSVD, PCA
from sklearn.svm import LinearSVC

import xgboost
# from mlxtend.classifier import EnsembleVoteClassifier 
import shap
import sklearn.neighbors



# Data Preparation

In [3]:
ls = [] 
directory = (r'./Dataset/CICIDS2017_cleaned_parquet/')
# for filename in os.listdir(r'./Dataset/MachineLearningCVE/'):
for filename in os.listdir(r'./Dataset/CICIDS2017_cleaned_parquet/'):
  if '.parquet' in  filename:
    print(filename)
    full_path = os.path.join(directory, filename)
    df = pd.read_parquet(full_path)
    ls.append(df)
    print(f'Shape: {df.shape}. Attack Type {df["Label"].unique()}')
    # break

DDoS-Friday-no-metadata.parquet
Shape: (221264, 78). Attack Type ['Benign', 'DDoS']
Categories (2, object): ['Benign', 'DDoS']
Bruteforce-Tuesday-no-metadata.parquet
Shape: (389714, 78). Attack Type ['Benign', 'FTP-Patator', 'SSH-Patator']
Categories (3, object): ['Benign', 'FTP-Patator', 'SSH-Patator']
Botnet-Friday-no-metadata.parquet
Shape: (176038, 78). Attack Type ['Benign', 'Bot']
Categories (2, object): ['Benign', 'Bot']
DoS-Wednesday-no-metadata.parquet
Shape: (584991, 78). Attack Type ['Benign', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']
Categories (6, object): ['Benign', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest', 'DoS slowloris', 'Heartbleed']
Benign-Monday-no-metadata.parquet
Shape: (458831, 78). Attack Type ['Benign']
Categories (1, object): ['Benign']
Infiltration-Thursday-no-metadata.parquet
Shape: (207630, 78). Attack Type ['Benign', 'Infiltration']
Categories (2, object): ['Benign', 'Infiltration']
Portscan-Friday-no-metadata.p

In [4]:
for df in ls:
  cols = df.columns.to_list()
  for i in range(len(cols)):
    cols[i] = cols[i].strip()
  df.columns = cols
df = pd.concat(ls)
pd.set_option('display.max_columns', None)
df.head(10)
# assert df.shape[1] == 79
print(df.shape)

from collections import Counter

Counter(df["Label"])

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Avg Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,3,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,6,109,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,1.100917e+05,18348.623850,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311523,9174.311523,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,6,52,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,2.307692e+05,38461.538460,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769531,19230.769531,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,6,34,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,3.529412e+05,58823.529410,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.765625,29411.765625,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,6,3,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
5,6,1022,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,1.174168e+04,1956.947162,1022.0,0.0,1022,1022,1022,1022.0,0.0,1022,1022,0,0.0,0.0,0,0,0,0,0,0,40,0,1956.947144,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
6,6,4,2,0,12,0,6,6,6.0,0.00000,0,0,0.0,0.0,3.000000e+06,500000.000000,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,500000.000000,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
7,6,42,1,1,6,6,6,6,6.0,0.00000,6,6,6.0,0.0,2.857143e+05,47619.047620,42.0,0.0,42,42,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,23809.523438,23809.523438,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,1,9.0,6.0,6.0,0,0,0,0,0,0,1,6,1,6,32,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
8,6,4,2,0,37,0,31,6,18.5,17.67767,0,0,0.0,0.0,9.250000e+06,500000.000000,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,1,0,0,0,40,0,500000.000000,0.000000,6,31,22.666666,14.433757,208.333328,0,1,0,0,1,0,0,0,0,34.0,18.5,0.0,0,0,0,0,0,0,2,37,0,0,946,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
9,6,3,2,0,37,0,31,6,18.5,17.67767,0,0,0.0,0.0,1.230000e+07,666666.666700,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,1,0,0,0,40,0,666666.687500,0.000000,6,31,22.666666,14.433757,208.333328,0,1,0,0,1,0,0,0,0,34.0,18.5,0.0,0,0,0,0,0,0,2,37,0,0,980,-1,1,20

(2313810, 78)


Counter({'Benign': 1977318,
         'DoS Hulk': 172846,
         'DDoS': 128014,
         'DoS GoldenEye': 10286,
         'FTP-Patator': 5931,
         'DoS slowloris': 5385,
         'DoS Slowhttptest': 5228,
         'SSH-Patator': 3219,
         'PortScan': 1956,
         'Web Attack � Brute Force': 1470,
         'Bot': 1437,
         'Web Attack � XSS': 652,
         'Infiltration': 36,
         'Web Attack � Sql Injection': 21,
         'Heartbleed': 11})

### Sampling and Label Encodings

In [5]:
#### Sample out benign data to a ratio of Benign : Malicious = 4:10
# print(f'df.shape {df.shape} before sampling out most of benign data')
# df = pd.concat([df[df['Label'] != 'BENIGN'], df[df['Label'] == 'BENIGN'].sample(frac=.1, random_state=42)]) # 20% of total is Malicious; Use only a sample of the BENIGN data
# print(f'df.shape {df.shape} after sampling out most of benign data')

#### Sample out Malicious data such that it would become an outlier
print(f'df.shape {df.shape} Before Sampling Out')
# df = pd.concat([df[df['Label'] == 'Benign'], df[df['Label'] != 'Benign'].sample(frac=0.1, random_state=42)]) 
print(f'df.shape {df.shape} after Sampling Out')

# #### filter out malicious data
# print(f'df.shape {df.shape} Before Sampling Out')
# df = df[df['Label'] == 'BENIGN']
# print(f'df.shape {df.shape} after Sampling Out')


df.shape (2313810, 78) Before Sampling Out
df.shape (2313810, 78) after Sampling Out


### Data Cleaning and Normalization

In [6]:
import gc 
gc.collect()
precleaned_df = df.copy()

0

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
df = precleaned_df.copy()
# labels = df['Label'].copy()
# strLabels = df['strLabel'].copy()
# df.drop(columns='Label', axis=1, inplace=True)
# df.drop(columns='strLabel', axis=1, inplace=True)


# Find and handle null values
null_counts = df.isnull().sum()
# Print the number of null values
print(f"{null_counts.sum()} null entries have been found in the dataset\n")
# Drop null values
df.dropna(inplace=True)          # or df_data = df_data.dropna()

# Find and handle duplicates
duplicate_count = df.duplicated().sum()
# Print the numer of duplicate entries
print(f"{duplicate_count} duplicate entries have been found in the dataset\n")
# Remove duplicates
df.drop_duplicates(inplace=True)  # or df_data = df_data.drop_duplicates()
# Display relative message
print(f"All duplicates have been removed\n")

# Reset the indexes
df.reset_index(drop=True, inplace=True)

# Inspect the dataset for categorical columns
print("Categorical columns:",df.select_dtypes(include=['object']).columns.tolist(),'\n')

# print(np.any(np.isnan(df)))
# print(np.any(np.isfinite(df)))

# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = RobustScaler()

# normalized_data = scaler.fit_transform(df[:])
normalized_df = pd.DataFrame(df, columns=df.columns)

# Last column turn to binary
# df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: 1 if x != 0 else 0).astype(int)
# normalized_df['Label'] = labels
# df['strLabel'] = strLabels
print(normalized_df.shape)



0 null entries have been found in the dataset

82004 duplicate entries have been found in the dataset

All duplicates have been removed

Categorical columns: ['Label'] 

(2231806, 78)


In [8]:
# 1.2.3 - Set up binary configuration

# Extract features as X
X = df.copy()
X = X.drop('Label', axis=1)

# Extract target as y
y = df['Label'].copy()

# Binarize labels
y = y.map({'Benign': 0}).fillna(1).astype(int)
y

0          0
1          0
2          0
3          0
4          0
          ..
2231801    0
2231802    0
2231803    0
2231804    0
2231805    0
Name: Label, Length: 2231806, dtype: int64

In [9]:

# X_train, X_temp, y_train, y_temp = train_test_split(normalized_df.drop(['Label'], axis=1), normalized_df['Label'], test_size=0.20, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

In [10]:
def extractAllSets(X,y,p_train,p_val,p_test,random_state, shuffle=True):

    X_train, X_temp, y_train, y_temp = train_test_split(X,y,                
                                                        stratify=y,
                                                        test_size=(1.0 - p_train),
                                                        random_state=random_state,
                                                        shuffle=shuffle)
                                                        # Enable/disable shuffling
    fraction = p_test / (p_val + p_test)
    X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,stratify=y_temp,
                                                      test_size=fraction,
                                                      random_state=random_state,
                                                      shuffle=shuffle)
                                                      # Enable/disable shuffling
    
    return X_train, X_val, X_test, y_train, y_val, y_test   
  
X_train, X_val, X_test, y_train, y_val, y_test = extractAllSets(X,y,0.5,0.1,0.4,random_state = 4022)
original_data = [X_train, X_val, X_test, y_train, y_val, y_test]

robust_scaler = RobustScaler()
#Validation for 
X_train_scaled = pd.DataFrame(robust_scaler.fit_transform(X_train), columns=X_train.columns)
X_val_scaled   = pd.DataFrame(robust_scaler.transform(X_val),       columns=X_val.columns)
X_test_scaled  = pd.DataFrame(robust_scaler.transform(X_test),      columns=X_test.columns)
y_train = y_train.reset_index(drop = True)
y_val = y_val.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)


In [11]:
# Assuming df is your DataFrame and you've already executed the train_test_split as shown earlier
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)

X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)  # Using X_test as validation data
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)  

X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)  # Using X_test as validation data
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)  

# X_full_tensor = torch.tensor(X.values, dtype=torch.float32)  # Using X_test as validation data
# y_full_tensor = torch.tensor(y.values, dtype=torch.float32)  

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

# full_dataset = TensorDataset(X_full_tensor, y_full_tensor)
# full_loader = DataLoader(full_dataset, batch_size=128, shuffle=False)




In [12]:
def get_score(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print(f"Accuracy: {accuracy:.4f}")
  print(f"Precision: {precision:.4f}")
  print(f"Recall: {recall:.4f}")
  print(f"F1 Score: {f1:.4f}")

### Get Feature Importance

In [82]:
# # Get Feature Importance
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()

# # fit random forest classifier on the training set
# rfc.fit(X_train_scaled., y_train)

# # extract important features
# score = np.round(rfc.feature_importances_,3)
# importances = pd.DataFrame({'feature':X_train_scaled.columns,'importance':score})
# importances = importances.sort_values('importance',ascending=False).set_index('feature')

# # plot importances
# plt.rcParams['figure.figsize'] = (11, 4)
# importances.plot.bar()
# selected_features=importances[:15].index.tolist()

ValueError: Found input variables with inconsistent numbers of samples: [111590, 1115903]

In [85]:
selected_features=['Avg Bwd Segment Size',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Packet Length Variance',
 'Packet Length Std',
 'Packet Length Mean',
 'Packet Length Max',
 'Subflow Bwd Bytes',
 'Bwd Packet Length Max',
 'Idle Max',
 'Avg Packet Size',
 'Fwd IAT Max',
 'Total Fwd Packets',
 'Subflow Fwd Packets',
 'Bwd Packets Length Total']

In [105]:
#Recursive feature elimination
# selected_features = importances[:15].index.tolist()

X_train_eng = X_train_scaled.loc[:,selected_features]
X_val_eng = X_val_scaled.loc[:,selected_features]
X_test_eng = X_test_scaled.loc[:,selected_features]

# Local Model Training

## Clustering

### PCA Dimensionality Reduction

In [106]:
pca = PCA(n_components=5, random_state=42)
pca_results = pca.fit_transform(X_train_scaled)
scaler = StandardScaler()
pca_results = scaler.fit_transform(pca_results)
pca_5d_df = pd.DataFrame(pca_results, columns=[f'PC{i+1}' for i in range(pca_results.shape[1])])
pca_5d_df['Label'] = y_train.values
pca_5d_df

,PC1,PC2,PC3,PC4,PC5,Label
0,-0.118789,3.010884,4.195642,-0.011059,-0.114978,0
1,-0.123536,-0.158339,-0.015182,-0.000998,-0.133270,1
2,-0.118755,0.443003,-0.419711,-0.001296,-0.145927,0
3,0.122674,2.089684,-1.591714,-0.002112,-0.187757,0
4,1.042697,-0.203882,0.041193,-0.003609,0.757005,0
...,...,...,...,...,...,...
1115898,-0.123536,-0.158339,-0.015182,-0.000995,-0.133267,0
1115899,-0.123536,-0.158340,-0.015181,-0.000994,-0.133214,0
1115900,-0.123536,-0.158340,-0.015179,-0.000995,-0.133075,0
1115901,-0.123536,-0.158339,-0.015182,-0.000996,-0.133270,0


### KNN Training

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

def knn_train(data, k_neighbors = 5):
  # y = data['Label'].values
  y = y_train
  # pca_results = data.drop("Label", axis=1, inplace=False)
  # pca_results = data
  # Initialize the KNN model. Let's start with K=5
  knn = KNeighborsClassifier(n_neighbors=k_neighbors)

  # Train the model
  knn.fit(data, y)
  # print(cross_val_score(knn,pca_results,y))
  return knn
# knn = knn_train(pca_5d_df.copy(),10)
knn = knn_train(X_train_eng.copy(),5)

In [108]:
# Predict labels for the test set
# X_test_pca = scaler.transform(pca.transform(test_X))
# pca_xtest_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])])

y_pred = knn.predict(X_test_eng)

# Evaluate the model
print("Shape of testing data = ", X_test_eng.shape)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
knn_labels = y_pred

Shape of testing data =  (892723, 15)
Accuracy: 0.9959113857265915

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    758126
           1       0.99      0.98      0.99    134597

    accuracy                           1.00    892723
   macro avg       0.99      0.99      0.99    892723
weighted avg       1.00      1.00      1.00    892723



## Other Algorithm for extracting boundary value
DBScan, Spectral Clustering, HDBScan
Find the Cluster transition
Try SVM!

## XGBoost Training

## Boundary Values Handling

### Get Boundary Values 

TODO: 
Convert to Inference 
Add votes data as new feature for xgboost
Save Data

In [112]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def get_sus(knn_model, X_new, y_test): #Record the Suspicious Data and compare it with the mismatch labels
    # Predict the class for a new data point
    X_new = X_new.copy()
    X_new = X_new.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)    
    # X_new = X_new.merge(y_test, right_index = True, left_index=True)
    # X_new_pca = scaler.transform(pca.transform(X_new))
    # pca_xnew_df = pd.DataFrame(X_new_pca, columns=[f'PC{i+1}' for i in range(X_new_pca.shape[1])])    
    predicted_class = knn_model.predict(X_new)
    
    # Get the k nearest neighbors
    distances, indices = knn_model.kneighbors(X_new)
    # Access the labels of the nearest neighbors
    neighbor_labels = knn_model.classes_[knn_model._y[indices].astype(int)]
    # Count the votes for each class
    sus_idx = []
    sus_datas = []
    count = 0
    for X_new_idx,neighbor in enumerate(neighbor_labels):
        
        votes = {label: 0 for label in knn_model.classes_}
        votes[0] = np.count_nonzero(neighbor == 0)
        votes[1] = np.count_nonzero(neighbor == 1)
        if votes[0] != 5 and votes[1] != 5: 
            print(X_new_idx, votes)
            sus_idx.append(X_new_idx)
            data_point = {
                'Index': X_new_idx,
                'Vote_0': votes[0],
                'Vote_1': votes[1],
                'Label': y_test.iloc[X_new_idx]
            }            
            sus_datas.append(data_point)
            
        
    
    # mismatch_indices = np.where(y_test.values != y_pred)[0]
    # print("Actual Mismatch Prediction = ", list(mismatch_indices))
    # print("Suspicious index", sus_df.index)
    print(sus_idx)
    sus_df = X_new.iloc[sus_idx].copy()
    sus_df = pd.merge(sus_df, pd.DataFrame(sus_datas), left_index=True, right_on = "Index")
    sus_df.drop(columns=['Index'], inplace=True)
    return sus_df


sus_df = get_sus(knn,X_train_eng, y_train)
sus_df

66 {0: 4, 1: 1}
117 {0: 4, 1: 1}
206 {0: 3, 1: 2}
225 {0: 4, 1: 1}
436 {0: 3, 1: 2}
483 {0: 4, 1: 1}
502 {0: 3, 1: 2}
570 {0: 4, 1: 1}
765 {0: 4, 1: 1}
866 {0: 4, 1: 1}
947 {0: 4, 1: 1}
1076 {0: 4, 1: 1}
1296 {0: 4, 1: 1}
1373 {0: 4, 1: 1}
1400 {0: 3, 1: 2}
1568 {0: 4, 1: 1}
1635 {0: 4, 1: 1}
1768 {0: 3, 1: 2}
1861 {0: 4, 1: 1}
1883 {0: 2, 1: 3}
1946 {0: 4, 1: 1}
2029 {0: 4, 1: 1}
2072 {0: 4, 1: 1}
2127 {0: 4, 1: 1}
2202 {0: 4, 1: 1}
2390 {0: 4, 1: 1}
2658 {0: 2, 1: 3}
2713 {0: 4, 1: 1}
2728 {0: 2, 1: 3}
2825 {0: 4, 1: 1}
2911 {0: 4, 1: 1}
3020 {0: 4, 1: 1}
3168 {0: 2, 1: 3}
3246 {0: 4, 1: 1}
3249 {0: 3, 1: 2}
3334 {0: 4, 1: 1}
3500 {0: 4, 1: 1}
3552 {0: 1, 1: 4}
3626 {0: 4, 1: 1}
3707 {0: 1, 1: 4}
3930 {0: 2, 1: 3}
3954 {0: 3, 1: 2}
3982 {0: 3, 1: 2}
4136 {0: 4, 1: 1}
4280 {0: 3, 1: 2}
4376 {0: 2, 1: 3}
4428 {0: 4, 1: 1}
4571 {0: 4, 1: 1}
4600 {0: 1, 1: 4}
4750 {0: 4, 1: 1}
4892 {0: 4, 1: 1}
5050 {0: 4, 1: 1}
5156 {0: 4, 1: 1}
5194 {0: 3, 1: 2}
5197 {0: 4, 1: 1}
5429 {0: 4, 1: 1}
5458

,Avg Bwd Segment Size,Bwd Packet Length Mean,Bwd Packet Length Std,Packet Length Variance,Packet Length Std,Packet Length Mean,Packet Length Max,Subflow Bwd Bytes,Bwd Packet Length Max,Idle Max,Avg Packet Size,Fwd IAT Max,Total Fwd Packets,Subflow Fwd Packets,Bwd Packets Length Total,Vote_0,Vote_1,Label
0,0.068883,0.068883,0.755271,74.750013,8.725738,9.152081,16.576320,2.093038,0.974964,0.0,8.690142,0.947661,6.8,6.8,2.093038,4,1,0
1,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000027,0.0,0.0,-0.061809,4,1,0
2,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,1.087939,0.2,0.2,-0.061809,3,2,1
3,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,0.947978,0.2,0.2,-0.061809,4,1,0
4,2.108535,2.108535,2.008873,2.384698,1.482748,1.833652,1.395863,1.084906,1.404149,0.0,1.829798,0.796045,0.8,0.8,1.084906,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12027,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000063,0.0,0.0,-0.061809,4,1,0
12028,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000120,0.4,0.4,-0.061809,3,2,0
12029,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,0.000032,0.2,0.2,-0.061809,4,1,0
12030,3.112016,3.112016,2.843428,4.101465,1.972760,1.878394,1.328816,0.583279,1.336910,0.0,2.221045,0.028604,0.0,0.0,0.583279,3,2,1


### XGBoost Boundary Training

In [113]:
sus_df1 = sus_df.copy()

In [120]:
### Finetuning with Boundary Value
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
p_train = 0.8
num_boost_round = 100

boundary_scaler = RobustScaler()
sus_df1.drop(columns=["Vote_0","Vote_1"])
sus_y = sus_df1['Label'].copy()
sus_y = sus_y.reset_index(drop=True)
sus_X = sus_df1.drop(columns="Label", axis = 1, inplace=False)
sus_X = pd.DataFrame(boundary_scaler.fit_transform(sus_X), columns=sus_X.columns)
sus_X_train, sus_X_valid, sus_y_train, sus_y_valid = train_test_split(sus_X,sus_y,                
                                                    stratify=sus_y,
                                                    test_size=(1.0 - p_train),
                                                    random_state=4022)

print(f'Size of X_train : {sus_X_train.shape}, Size of X_valid L {sus_X_valid.shape}')
dsus_train = xgb.DMatrix(data=sus_X_train, label=sus_y_train) 
dsus_valid = xgb.DMatrix(data=sus_X_valid, label=sus_y_valid) 


# params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
# finetuned_xgb = xgb.train(params, dsus_train, num_boost_round, evals=[(dsus_valid, 'validation')], early_stopping_rounds=10, verbose_eval=0, xgb_model=bst)


params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
boundary_xgb = xgb.train(params, dsus_train, num_boost_round, evals=[(dsus_valid, 'validation')], early_stopping_rounds=10, verbose_eval=0)


,Avg Bwd Segment Size,Bwd Packet Length Mean,Bwd Packet Length Std,Packet Length Variance,Packet Length Std,Packet Length Mean,Packet Length Max,Subflow Bwd Bytes,Bwd Packet Length Max,Idle Max,Avg Packet Size,Fwd IAT Max,Total Fwd Packets,Subflow Fwd Packets,Bwd Packets Length Total,Label
0,0.068883,0.068883,0.755271,74.750013,8.725738,9.152081,16.576320,2.093038,0.974964,0.0,8.690142,0.947661,6.8,6.8,2.093038,0
1,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000027,0.0,0.0,-0.061809,0
2,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,1.087939,0.2,0.2,-0.061809,1
3,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,0.947978,0.2,0.2,-0.061809,0
4,2.108535,2.108535,2.008873,2.384698,1.482748,1.833652,1.395863,1.084906,1.404149,0.0,1.829798,0.796045,0.8,0.8,1.084906,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12027,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000063,0.0,0.0,-0.061809,0
12028,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,-0.000120,0.4,0.4,-0.061809,0
12029,-0.360763,-0.360763,0.000000,-0.008793,-0.095672,-0.403442,-0.085592,-0.061809,-0.081545,0.0,-0.488704,0.000032,0.2,0.2,-0.061809,0
12030,3.112016,3.112016,2.843428,4.101465,1.972760,1.878394,1.328816,0.583279,1.336910,0.0,2.221045,0.028604,0.0,0.0,0.583279,1


Size of X_train : (9625, 17), Size of X_valid L (2407, 17)


In [121]:
sus_X

,Avg Bwd Segment Size,Bwd Packet Length Mean,Bwd Packet Length Std,Packet Length Variance,Packet Length Std,Packet Length Mean,Packet Length Max,Subflow Bwd Bytes,Bwd Packet Length Max,Idle Max,Avg Packet Size,Fwd IAT Max,Total Fwd Packets,Subflow Fwd Packets,Bwd Packets Length Total,Vote_0,Vote_1
0,0.429646,0.429646,0.755271,74.758807,8.821410,647.829083,3893.333333,2.154847,1.056509,0.0,561.451864,0.962600,33.0,33.0,2.154847,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.005335,-1.0,-1.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.105875,0.0,0.0,0.000000,-1.0,1.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.962924,0.0,0.0,0.000000,0.0,0.0
4,2.469298,2.469298,2.008873,2.393491,1.578421,151.666669,346.166667,1.146714,1.485694,0.0,141.818186,0.807744,3.0,3.0,1.146714,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.005372,-1.0,-1.0,0.000000,0.0,0.0
12028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.005430,1.0,1.0,0.000000,-1.0,1.0
12029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.005274,0.0,0.0,0.000000,0.0,0.0
12030,3.472779,3.472779,2.843428,4.110258,2.068432,154.699997,330.500000,0.645088,1.418455,0.0,165.750000,0.023908,-1.0,-1.0,0.645088,-1.0,1.0


In [122]:

# # Before
# predictions = bst.predict(dsus_valid)
# predictions_binary = (predictions > 0.5).astype(int)
# print(get_score(sus_y_valid, predictions_binary))

# # Finetuned
# predictions = finetuned_xgb.predict(dsus_valid)
# predictions_binary = (predictions > 0.5).astype(int)
# print(get_score(sus_y_valid, predictions_binary))

# # Boundary
predictions = boundary_xgb.predict(dsus_valid)
predictions_binary = (predictions > 0.5).astype(int)
print(get_score(sus_y_valid, predictions_binary))

Accuracy: 0.7765
Precision: 0.7096
Recall: 0.4452
F1 Score: 0.5471
None


## One Layer XGBoost

In [99]:


# Convert pandas DataFrame to DMatrix
dtrain = xgb.DMatrix(data=X_train_eng, label=y_train)  # Assuming y_train is your training labels
dvalid = xgb.DMatrix(data = X_val_eng, label=y_val)
dtest = xgb.DMatrix(data=X_test_eng, label=y_test)    # Assuming y_test is your testing labels

# Define parameters for XGBoost
params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}

# Number of boosting rounds
num_boost_round = 100

# Train XGBoost
bst = xgb.train(params, dtrain, num_boost_round, evals=[(dvalid, 'validation')], early_stopping_rounds=10, verbose_eval=0)

# bst = XGBClassifier(**params)
# bst.fit(X_train_scaled, y_train)


# Make predictions
# predictions = bst.predict(dtest)
predictions = bst.predict(dtest)
predictions_binary = (predictions > 0.5).astype(int)

# Evaluate predictions using accuracy_score
print(get_score(y_test, predictions_binary))


Accuracy: 0.9955
Precision: 0.9937
Recall: 0.9764
F1 Score: 0.9850
None


## Inference

In [20]:
def knn_inference(knn_model, X_new): #Record the Suspicious Data and compare it with the mismatch labels
    # Predict the class for a new data point
    X_new = X_new.copy()
    X_new = X_new.reset_index(drop=True)
    X_new_pca = scaler.transform(pca.transform(X_new))
    pca_xnew_df = pd.DataFrame(X_new_pca, columns=[f'PC{i+1}' for i in range(X_new_pca.shape[1])])    
    predicted_class = knn_model.predict(pca_xnew_df)
    
    # Get the k nearest neighbors
    distances, indices = knn_model.kneighbors(pca_xnew_df)
    # Access the labels of the nearest neighbors
    neighbor_labels = knn_model.classes_[knn_model._y[indices].astype(int)]
    # Count the votes for each class
    sus_idx = []
    train_idx = []
    sus_datas = []
    prediction_rec = []
    count = 0
    for X_new_idx,neighbor in enumerate(neighbor_labels):
        
        votes = {label: 0 for label in knn_model.classes_}
        votes[0] = np.count_nonzero(neighbor == 0)
        votes[1] = np.count_nonzero(neighbor == 1)
        if votes[0] != 10 and votes[1] != 10: 
            sus_idx.append(X_new_idx)
            data_point = {
                'Index': X_new_idx,
                'Vote_0': votes[0],
                'Vote_1': votes[1],
            }            
            sus_datas.append(data_point)
            prediction_rec.append("Pending")
        else:
            train_idx.append(X_new_idx)
            if votes[0] == 10:
                prediction_rec.append("Benign")
            else:
                prediction_rec.append("Malicious")
    sus_df = X_new.iloc[sus_idx].copy()
    sus_data_df = pd.DataFrame(sus_datas)
    sus_data_df.set_index("Index", inplace=True)
    train_df = X_new.iloc[train_idx].copy()
    train_df["Label"] = np.array(prediction_rec)[train_idx] #where it is not equal to "Pending"
    sus_df = pd.merge(sus_df, sus_data_df, how='left', left_index=True, right_index=True)
    return train_df, sus_df, prediction_rec


In [21]:
# Perform KNN Inference X_Val_Scaled
train_df, sus_df, prediction_rec = knn_inference(knn, X_test_scaled)
knn_prediction_df = pd.DataFrame(prediction_rec, columns=["Prediction"])

# Save Training Data for KNN (Remember to use unscaled dataset instead)
# if os.path.exists("knn_train1.csv"):
#   train_df.iloc[:10].to_csv("knn_train1.csv", mode="a", index=False, header=False)
# else:
#   train_df.iloc[:10].to_csv("knn_train1.csv", index=False)
  
# Evaluate 1st Layer Accuracy
knn_eval_pred_df = knn_prediction_df.loc[knn_prediction_df["Prediction"] != "Pending"]
print("Layer 1 Performance")
get_score(knn_eval_pred_df["Prediction"].map({"Benign":0, "Malicious":1}), y_test.iloc[knn_eval_pred_df.index])
  

Layer 1 Performance
Accuracy: 0.9958
Precision: 0.9774
Recall: 0.9911
F1 Score: 0.9842


In [22]:
sus_df = pd.DataFrame(boundary_scaler.transform(sus_df), columns = sus_df.columns, index = sus_df.index)
sus_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Avg Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Vote_0,Vote_1
4,0.0,-0.124366,-0.6,-0.6,-0.263848,-0.060575,-0.345708,0.000000,-0.219570,-0.080148,-0.132981,0.065217,-0.093331,-0.185790,15.812003,100.805688,-0.130291,-0.140225,-0.179497,0.811321,-0.035957,-0.042299,-0.017548,-0.021565,0.233100,-0.035550,-0.060922,-0.037100,-0.037146,-0.06,1.0,0.0,0.0,0.0,-0.46875,-0.575,132.518359,161.414019,0.000000,-0.317101,-0.276519,-0.290181,-0.090054,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.269639,-0.219570,-0.093331,0.0,0.0,0.0,0.0,0.0,0.0,-0.6,-0.263848,-0.6,-0.060575,-0.279042,-0.966245,-0.50,1.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.4,-0.4
12,0.0,-0.086604,-0.4,0.2,-0.271137,0.939943,-0.371230,0.000000,-0.284805,-0.141273,1.535426,0.000000,1.527577,2.160367,1.700220,0.055000,-0.076331,-0.075464,-0.122933,0.415094,-0.034981,-0.039275,-0.015531,-0.020533,2.305361,0.144451,0.266590,0.310877,0.220613,0.86,0.0,0.0,0.0,0.0,-0.40625,0.000,0.033456,0.251276,0.000000,1.010974,1.372536,1.171957,2.069892,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.464703,-0.284805,1.527577,0.0,0.0,0.0,0.0,0.0,0.0,-0.4,-0.271137,0.2,0.939943,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,-1.0,1.0
21,1.0,0.009294,-0.6,-0.4,-0.163265,-0.040124,-0.292343,1.179487,0.109785,-0.165427,-0.096089,1.173913,-0.012432,-0.168939,-0.041550,-0.026339,0.315724,0.174553,0.008087,701.132075,0.128819,0.997725,-0.017548,0.171227,4492.379953,0.589934,4.491343,-0.037100,0.877235,40445.58,0.0,0.0,0.0,0.0,-0.56250,-0.375,-0.023385,-0.000919,1.179487,-0.293324,-0.195806,-0.274809,-0.089489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.176802,0.109785,-0.012432,0.0,0.0,0.0,0.0,0.0,0.0,-0.6,-0.163265,-0.4,-0.040124,-0.283015,-0.970464,-0.25,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,0.0
29,0.0,3.943981,4.2,4.4,0.985423,1.931228,0.097448,0.000000,-0.041491,0.157422,0.664903,0.000000,0.524465,0.588720,-0.022561,-0.034006,0.668262,2.340521,5.656395,-0.037736,5.325071,1.310652,3.522406,5.860813,-0.132867,19.326408,5.579733,14.343980,26.549460,0.00,0.0,0.0,0.0,0.0,5.68750,4.550,-0.030944,-0.019332,0.000000,0.315043,0.286661,0.190427,0.150223,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.261837,-0.041491,0.524465,0.0,0.0,0.0,0.0,0.0,0.0,4.2,0.985423,4.4,1.931228,0.725690,-0.451477,1.25,1.0,176.725642,0.0,176.667342,179.058674,5.88,0.0,5.88,5.904030,-0.4,0.4
35,0.0,0.109138,0.0,-0.8,-0.265306,-0.061092,-0.403712,0.153846,-0.310263,-0.195090,-0.134697,0.000000,-0.097533,-0.185790,-0.055460,-0.030146,0.454097,0.402220,0.177459,4.754717,0.271641,0.442989,0.526509,0.338231,0.354312,-0.035550,-0.060922,-0.037100,-0.037146,-0.06,0.0,0.0,0.0,0.0,-0.18750,-0.700,-0.017697,-0.033061,0.153846,-0.322817,-0.290748,-0.300929,-0.090169,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.

In [23]:
# XGBoost Inference
dinference = xgb.DMatrix(data=sus_df)
predictions = boundary_xgb.predict(dinference)
predictions_binary = np.where(predictions > 0.5, "Malicious", "Benign")
predictions_binary = pd.DataFrame({
    'Prediction': predictions_binary
}, index=sus_df.index)
print(predictions_binary.index)

Index([     4,     12,     21,     29,     35,     41,     44,     45,     68,
           70,
       ...
       892660, 892661, 892667, 892668, 892679, 892680, 892685, 892699, 892704,
       892717],
      dtype='int64', length=75106)


In [24]:
# Final Prediction
# knn_prediction_df.loc[knn_prediction_df['Prediction'] == 'Pending', 'Prediction'] = predictions_binary
knn_prediction_df.update(predictions_binary)

knn_prediction_df["Prediction"]=knn_prediction_df["Prediction"].map({"Benign": 0}).fillna(1).astype(int)

knn_prediction_df

,Prediction
0,0
1,0
2,0
3,0
4,0
...,...
892718,0
892719,1
892720,0
892721,0


In [25]:
# Get Layer 2 Score
get_score(y_test.iloc[predictions_binary.index], predictions_binary["Prediction"].map({"Benign":0, "Malicious":1}))

Accuracy: 0.9932
Precision: 0.9907
Recall: 0.9898
F1 Score: 0.9902


In [26]:
# Get Overall Score
get_score(knn_prediction_df, y_test)

Accuracy: 0.9956
Precision: 0.9798
Recall: 0.9910
F1 Score: 0.9854


# Global Model

## Grid Search for the 3 Best Models

In [27]:
y_glob_train

NameError: name 'y_glob_train' is not defined

In [ ]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
y_global = train_df["Label"].copy()
y_global = y_global.map({"Benign":0}).fillna(1).astype(int)

X_global = train_df.drop(columns=["Label"],axis=1, inplace=False)
X_glob_train, X_glob_valid, y_glob_train, y_glob_valid = train_test_split(X_global, y_global, stratify=y_global, test_size=0.2, shuffle=True, random_state=4022)

dglobal_train = xgb.DMatrix(data = X_glob_train, label=y_glob_train)
dglobal_valid = xgb.DMatrix(data = X_glob_valid, label=y_glob_valid)

def objective(max_depth, min_child_weight, subsample, colsample_bytree, num_boost_round, learning_rate):
    params = {
        'max_depth': int(max_depth),  # Ensuring integer type
        'min_child_weight': int(min_child_weight),  # Ensuring integer type
        'subsample': float(subsample),  # Ensuring float type
        'colsample_bytree': float(colsample_bytree),  # Ensuring float type
        'learning_rate': float(learning_rate),  # Ensuring float type
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
    }
    
    # dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
    
    # Ensure num_boost_round is an integer
    cv_results = xgb.cv(
        params,
        dglobal_train,
        num_boost_round=int(num_boost_round),  # Correct casting to integer
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )
    
    loss = cv_results['test-logloss-mean'].min()
    return -loss


# Define the parameter bounds
pbounds = {
    'max_depth': (3, 12),
    'min_child_weight': (1, 6),
    'subsample': (0.7, 1.0),
    'colsample_bytree': (0.7, 1.0),
    'num_boost_round': (100, 300),  # Change from n_estimators to num_boost_round
    'learning_rate': (0.01, 0.2)
}

optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=10,
    n_iter=20,
)

print(optimizer.max)


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | num_bo... | subsample |
-------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# params = optimizer.max["params"]
params = {
    'max_depth': int(params["max_depth"]),  # Ensuring integer type
    'min_child_weight': int(params["min_child_weight"]),  # Ensuring integer type
    'subsample': float(params["subsample"]),  # Ensuring float type
    'colsample_bytree': float(params["colsample_bytree"]),  # Ensuring float type
    'learning_rate': float(params["learning_rate"]),  # Ensuring float type
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
}
global_xgb = xgb.train(params, dglobal_train, num_boost_round, evals=[(dglobal_valid, 'validation')], early_stopping_rounds=10, verbose_eval=0)

predictions = global_xgb.predict(dglobal_valid)
predictions_binary = (predictions > 0.5).astype(int)
print(get_score(y_glob_valid, predictions_binary))

Accuracy: 0.9965
Precision: 0.9901
Recall: 0.9844
F1 Score: 0.9872
None


In [ ]:
params = {'colsample_bytree': 0.7558780634133012, 'learning_rate': 0.07565653813817907, 'max_depth': 6.570907268076029, 'min_child_weight': 3.6940836700167847, 'num_boost_round': 183.83890288065896, 'subsample': 0.9055658501190278}
print(optimizer.max["params"])
# {'colsample_bytree': 0.7558780634133012, 'learning_rate': 0.07565653813817907, 'max_depth': 6.570907268076029, 'min_child_weight': 3.6940836700167847, 'num_boost_round': 183.83890288065896, 'subsample': 0.9055658501190278}


KeyError: 'params'

100

## Inference and Labeling

In [ ]:
#Load the Model
global_xgb
#Read the csv file
knn_boundary_df = pd.read_csv("knn_train1.csv")
knn_boundary_df.drop(columns=["Label"], axis=1, inplace=True)
dinference = xgb.DMatrix(data = knn_boundary_df)
#Predict and Label
predictions = global_xgb.predict(dinference)
predictions_binary = np.where(predictions > 0.5, "Malicious", "Benign")
knn_boundary_df["Label"] = predictions_binary


In [ ]:
knn_boundary_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Avg Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,19.332354,3.2,2.6,2.773006,0.577286,1.438849,-0.153846,0.320895,1.508812,0.618741,0.000000,0.097897,0.698087,-0.138742,-0.032091,3.859582,2.194232,1.739905,0.961039,20.311294,5.903558,2.329423,1.833868,78.470588,411.107976,166.690031,72.672812,59.595087,79.895833,0.0,0.0,0.0,0.0,2.333333,3.166667,-0.022700,-0.195937,-0.153846,0.612696,0.093061,0.487912,0.318518,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,-0.002881,0.320895,0.097897,0.0,0.0,0.0,0.0,0.0,0.0,3.2,2.773006,2.6,0.577286,0.971195,0.525424,5.333333,0.000000,107552.773438,610359.625000,1027704.5,15486.5,1.942552,59980.835938,1.946092,1.932239,Benign
1,0.0,-0.012014,0.0,-0.2,0.631902,-0.059876,0.827338,0.000000,2.238806,2.016850,-0.077253,0.064516,-0.339953,0.000000,3.035192,0.383287,-0.030190,-0.006944,-0.009975,0.389610,0.000640,0.003451,0.000000,0.000691,64.897059,-0.000015,-0.000083,0.000000,-0.000024,-0.062500,1.0,0.0,0.0,0.0,-0.181818,-0.366667,0.358759,2.545856,0.000000,0.109843,0.400295,0.265928,0.116895,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.529504,2.238806,-0.339953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.631902,-0.2,-0.059876,-0.021360,0.004237,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Benign
2,1.0,-0.004663,0.0,0.0,-0.012270,0.113244,-0.032374,0.743590,-0.111940,0.000000,0.110873,2.892473,0.581776,0.000000,0.468529,0.017685,-0.015196,0.004784,-0.002167,-0.025974,-0.000127,-0.000326,0.000000,-0.000118,-0.014706,-0.000007,-0.000042,0.000000,-0.000012,-0.020833,0.0,0.0,0.0,0.0,0.121212,0.000000,0.011570,0.297128,0.743590,0.106277,0.334793,0.203825,0.077436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388011,-0.111940,0.581776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.012270,0.0,0.113244,-0.028805,0.000000,0.000000,1.666667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Benign
3,1.0,-0.007467,-0.2,-0.2,-0.116564,-0.039701,-0.028777,0.769231,-0.089552,0.000000,-0.032904,0.731183,-0.122664,0.000000,0.019434,0.005984,0.001429,-0.008375,-0.005143,415.350649,-0.000127,-0.000328,0.000000,-0.000119,-0.044118,-0.000015,-0.000083,0.000000,-0.000024,-0.062500,0.0,0.0,0.0,0.0,-0.333333,-0.366667,0.003512,0.181261,0.769231,-0.037090,-0.135986,-0.052634,-0.007029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.107439,-0.089552,-0.122664,0.0,0.0,0.0,0.0,0.0,0.0,-0.2,-0.116564,-0.2,-0.039701,-0.028805,0.000000,-0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Benign
4,0.0,0.358342,0.4,-0.4,-0.153374,-0.061829,-0.136691,0.000000,-0.761194,0.000000,-0.081545,0.000000,-0.360981,0.000000,-0.139448,-0.031175,0.763838,0.595708,0.382991,-0.038961,0.390476,0.641424,0.674337,0.412343,-0.029412,-0.000015,-0.000083,0.000000,-0.000024,-0.062500,0.0,0.0,0.0,0.0,0.121212,-0.533333,-0.021332,-0.197486,0.000